This works the same way as sort_images_by_proposid.py, please check out that documentation too

In [ ]:
import os
from astropy.io import fits
import argparse

parser=argparse.ArgumentParser()
parser.add_argument('-dir', type=str, default='./', help='give root direcory')
args=parser.parse_args()
indir=args.dir
images=os.listdir(indir)
for image in images:
    if not image[-5:]=='.fits':
        print '>>> image: \t %s does not end with an .fits, it is no fits file\n'%(image)
        continue
    print image
    try:
        hdulist=fits.open(indir+image)
    except IOError:
        print '>>> image: \t %s is not a .fits file or has no header\n'%(image)
        continue

see sort_images_by_proposid.py.ipynb

In [ ]:
    try:
        filter_from_image=hdulist[0].header['FILTER']
        fnum=0
    except KeyError:
        filter_from_image=None
    if filter_from_image is None:
        try:
            filter_from_image=hdulist[0].header['FILTER2']
            fnum=2
        except KeyError:
            pass

        if filter_from_image=='CLEAR2L':
            try: 
                filter_from_image=hdulist[0].header['FILTER1']
                fnum=1
            except KeyError:
                pass    
        if filter_from_image=='CLEAR1L':
            print '>>>  image: \t %s does not have a Filter'%(image)
            continue

        fits.setval(indir+image, 'FILTER', value=filter_from_image)

Ckeck if Keyword Filter is in the header, this is sometimes not the case
If not found check other keywords FILTER1 and FILTER2
Take the Keyword which is not CLEAR1/2L
Set FILTER keyword respectively

In [ ]:
    try:
        print 'using filter %i'%(fnum)
    except:
        print 'image: \t %s does not have a Filter'%(image)
        continue    

    outdir=indir+filter_from_image+'/'
    print 'image: \t %s \tFilter \t %s \t from Filter %i' %(image, filter_from_image, fnum)
    print 'indir: \t %s' %(indir)
    print 'outdir:\t %s' %(outdir)
    print '\n'
    try:
        os.rename(indir+image, outdir+image)
    except OSError:
        print '>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>\ncreating directory /%s/\n>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>'%(filter_from_image)
        os.mkdir(outdir)
        os.rename(indir+image, outdir+image)

Shows the result it found for FILTER if it has found sth
Defines Outdir eg ./F225W/
Moves Files, if it does not work creates Directory and tries again